In [24]:
#Diversos
import pandas as pd
from IPython.display import clear_output
import ToolBox.kit_function as kit

#WebDriver
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

#Ações
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
#from ast import Break

# Exceções
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException as Timeout
from selenium.common.exceptions import ElementNotInteractableException as NotInteract
from selenium.common.exceptions import ElementClickInterceptedException as NotAvailibleElement
from selenium.common.exceptions import NoSuchWindowException as CrashJanela
from selenium.webdriver.support import expected_conditions as EC

In [25]:
# Inicializa a instância do EDGE
service = Service()

options = webdriver.EdgeOptions()
options.add_argument("-inprivate")
driver = webdriver.Edge(service=service, options=options)
driver.maximize_window()

url = 'https://toolsworldb2b.meuspedidos.com.br/'

driver.get(url)

In [26]:
sheetCategorias = 'Links/CategoriasMTX.xlsx'
columns = ['Link']
categoriasLinks = kit.readExcel(sheetCategorias,columns)

In [ ]:
for link in categoriasLinks['Link']:
    print(link)

In [4]:
#Captura o título da categoria
cssSeletorTitle = '#catalogWrapper > div:nth-child(2) > div.Catalog__titleB2bContainer___JtAi4 > div > div.Catalog__titleB2bInner___G8ViQ'

titleCategoria = driver.find_element(By.CSS_SELECTOR , cssSeletorTitle).text
print(titleCategoria)

ABRAÇADEIRA


In [30]:
#Scroll No menu de categorias para carregar elementos
footer = driver.find_element(By.TAG_NAME, "footer")
scroll_origin = ScrollOrigin.from_element(footer, 0, -50)
ActionChains(driver)\
    .scroll_from_origin(scroll_origin, 0, 8000)\
    .perform()

In [ ]:
#https://toolsworldb2b.meuspedidos.com.br/?categoria=1740255&representada=448126

In [33]:
#Verifica a qtd de itens na pagina
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'catalog-item-b2b-title')))
itens = driver.find_elements(By.CLASS_NAME, 'catalog-item-b2b-title')
print('Iniciando varredura, quantidade inicial:', len(itens))

#Scrolla se necessario para baixo e abre mais itens na pagina
try:
    while True:
        verProdutos = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[class='b2b-button Catalog__links___4450R Catalog__buttonSeeMoreProducts___kZGUm Button__button___Oss6- Button__pointer___mYfJr Button__info___4h9Qg Button__medium___FQv5P Button__buttonInfo___aqiZj custom-brand-full-b2b']")))
        verProdutos.click()
except Timeout:
        print('Verificação Terminada. Total: ')
        #Break()
        
head = driver.find_element(By.TAG_NAME, "footer")
delta_x = head.rect['x']
ActionChains(driver)\
    .scroll_by_amount(delta_x, -50000)\
    .perform()

#Seta a quantidade de itens na pagina
itens = driver.find_elements(By.CLASS_NAME, 'catalog-item-b2b-title')
print(len(itens) , 'Itens')

Iniciando varredura, quantidade inicial: 48
Verificação Terminada. Total: 
177 Itens
